# Neural Network Basic
- In this file we can tell the basics of neurak network.
- Means that we can see how we can build our neural network.

# Step
- Get data
- Build Model
- Train Model
- Evulate Model
- Save the Model

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader # for data loading
from torchvision import datasets # prebuild datasets
import torchvision
from torchvision.transforms import ToTensor # Transformations

# Load Dataset
- We will use the prebuild dataset from troch so that we can train our custom model.
- We will use the fashion Mnist Dataset which have 10 classes.

In [2]:
# we wil use pre build image net dataset
train_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# Test data
test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.2MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 209kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.85MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 4.36MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
train_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

# Observation
- we can get the data successfully.
- Now we can make a dataloader so that we can load the data efficently.

In [5]:
# Train Loader
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)

# Test Loader
test_loader = DataLoader(test_data, batch_size=8, shuffle=False)

In [6]:
# View the barch
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([8, 1, 28, 28])
torch.Size([8])


# Build Model
- we know that cnn is best on image data.
- But can make a simple Sequentail Model and domot use `CNN`.
- After training the simle network we will clear about basic so we can make `CNN`
- Get the data ==> `Flatten` the input.
- Pass the `flatten input` to the `linear model`
- Linear model with expect input shape and output shape.
- In this our image will be the shape of 28*28 so our model input size will be 28*28 and output node will be depend on us.
- **example** = `nn.Linear(28*28 ,512)`

In [7]:
class MyCustomModel(nn.Module):
    def __init__(self):
        super(MyCustomModel,self).__init__()
        self.flatten = nn.Flatten()

        self.linear_seq = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x= self.flatten(x)
        x= self.linear_seq(x)
        return x

# Observation
- We can make a simple Model with 2 hidden layer.
- Our Goal is not make the perfect model.

In [17]:
# Set Optimizer and Loss
model = MyCustomModel()

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

model, loss_fn, optimizer

(MyCustomModel(
   (flatten): Flatten(start_dim=1, end_dim=-1)
   (linear_seq): Sequential(
     (0): Linear(in_features=784, out_features=512, bias=True)
     (1): ReLU()
     (2): Linear(in_features=512, out_features=512, bias=True)
     (3): ReLU()
     (4): Linear(in_features=512, out_features=10, bias=True)
   )
 ),
 CrossEntropyLoss(),
 SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     fused: None
     lr: 0.0001
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 ))

# make Training Loop

In [18]:
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [19]:
model = model.to(device)
epochs = 10
for epoch in range(epochs):
    train_loss = 0
    num_correct = 0
    num_samples = 0
    for image, label in tqdm(train_loader):
        image = image.to(device)
        label = label.to(device)

        # do forward pass
        output = model(image)

        # Calculate the loss
        loss = loss_fn(output,label)

        # Set the optimizer to zero_grad
        optimizer.zero_grad()

        # do backward
        loss.backward()

        optimizer.step()

        train_loss +=loss.item()

        # calculate the accuracy
        _, prediction = output.max(1)
        num_correct += (prediction == label).sum()
        num_samples += prediction.size(0)

    # calculate the accuracy
    accuracy = num_correct/num_samples
    print(f"Epoch: {epoch + 1}/{epochs}, Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {accuracy * 100:.2f}%")

100%|██████████| 7500/7500 [00:34<00:00, 220.15it/s]


Epoch: 1/10, Train Loss: 2.2373, Train Accuracy: 19.43%


100%|██████████| 7500/7500 [00:34<00:00, 220.00it/s]


Epoch: 2/10, Train Loss: 2.0841, Train Accuracy: 42.00%


100%|██████████| 7500/7500 [00:33<00:00, 227.16it/s]


...

Epoch: 8/10, Train Loss: 0.9667, Train Accuracy: 66.60%


100%|██████████| 7500/7500 [00:36<00:00, 207.87it/s]


Epoch: 9/10, Train Loss: 0.9089, Train Accuracy: 67.74%


100%|██████████| 7500/7500 [00:34<00:00, 220.33it/s]

Epoch: 10/10, Train Loss: 0.8644, Train Accuracy: 68.71%



# Model Evulation

In [21]:
model.eval()
with torch.no_grad():
    test_loss = 0
    num_correct = 0
    num_samples = 0
    for image, label in tqdm(test_loader):
        image = image.to(device)
        label = label.to(device)

        # do forward pass
        output = model(image)

        # calculate the loss
        loss = loss_fn(output,label)

        test_loss += loss.item()

        # calculate the accuracy
        _,prediction = output.max(1)
        num_correct += (prediction == label).sum()
        num_samples += prediction.size(0)

    accuracy = num_correct/num_samples
    print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {accuracy * 100:.2f}%")

100%|██████████| 1250/1250 [00:02<00:00, 447.69it/s]

Test Loss: 0.8586, Test Accuracy: 67.56%


# Observation
- We see that we can train the model in 10 epochs and we will get the 67% accuracy in this dataset with simple sequential model.